In [ ]:
import os
    
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle
from matplotlib import animation
from IPython.display import HTML
import seaborn as sns
from PIL import Image

from tqdm import tqdm
from scipy.spatial.distance import pdist, cdist, squareform
from scipy.stats import pearsonr, spearmanr, entropy, rankdata
from sklearn.manifold import MDS
from scipy.ndimage import gaussian_filter1d

import manifold_dynamics.tuning_utils as tut

RAND = 0
RESP = (50,220)
BASE = (-50,0)
ONSET = 50
RESP = slice(ONSET + RESP[0], ONSET + RESP[1])
BASE = slice(ONSET + BASE[0], ONSET + BASE[1])

In [ ]:
DATA_DIR = '../../datasets/NNN/'
IMG_DIR = '../../datasets/NNN/NSD1000_LOC'

CAT = 'face'
dat = pd.read_pickle(os.path.join(DATA_DIR, (f'{CAT}_roi_data.pkl')))
ROI_LIST = list(dat['roi'].unique())
print(f'Unique face ROIs: {ROI_LIST}')

with open(f'../../datasets/NNN/{CAT}_mins.pkl', 'rb') as f:
    mins = pickle.load(f)
    
SCALES = {k: v[0] for k,v in mins.items()}

SAVE_DIR = '../../../buckets/manifold-dynamics/time-time/'
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

In [ ]:
len(dat[dat['roi'] == 'Unknown_19_F'])

In [ ]:
ROI = 'Unknown_19_F' # Unknown_19_F, MF1_7_F, MF1_8_F, MF1_9_F, AF3_18_F, MB1_3_B
MODE = 'top'
scale = -1
vmin, vmax= 0, 1
di = {-1: 'shuff', 1072: 'all'}

Rt, _ = tut.static_rdm(dat, ROI, mode=MODE, scale=scale,
                          tstart=0, tend=450)
        
fig, ax = plt.subplots(figsize=(2, 2))
sns.heatmap(Rt, ax=ax, square=True, 
            vmin=vmin, vmax=vmax, cbar=False)

ax.set_xticks([])
ax.set_yticks([])

ax.set_xlabel('Time')
ax.set_ylabel('Time')

ymin, ymax = ax.get_ylim()
ax.vlines(x=100, ymin=ymin, ymax=ymax, color='red', linestyle='--')
ax.vlines(x=270, ymin=ymin, ymax=ymax, color='red', linestyle='--')

ax.hlines(y=100, xmin=ymin, xmax=ymax, color='red', linestyle='--')
ax.hlines(y=270, xmin=ymin, xmax=ymax, color='red', linestyle='--')


# plt.savefig(os.path.join(SAVE_DIR, f'{ROI}_{di[scale]}.png'), format='png', dpi=300, transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
ROI = 'MF1_8_F' # Unknown_19_F, MF1_7_F, MF1_8_F, MF1_9_F, AF3_18_F, MB1_3_B
MODE = 'top'
scale = SCALES[ROI]
# 100 t0, 200 window for medial
# 150 to, 300 window for anterior
t0 = 0 
window = 450 #

Rt, _ = tut.static_rdm(dat, ROI, mode=MODE, scale=scale,
                  tstart=t0, tend=t0 + window)

print(tut.ED2(Rt))
# np.fill_diagonal(Rt, np.nan)

Ra, _ = tut.static_rdm(dat, ROI, mode=MODE, scale=1072,
                  tstart=t0, tend=t0 + window)
print(tut.ED2(Ra))

Rss = []
for i in tqdm(range(50)):
    Rs, _ = tut.static_rdm(dat, ROI, mode='shuff', scale=scale,
                      tstart=t0, tend=t0 + window, random_state=i)
    Rss.append(Rs)
Rs_EDs = np.array([tut.ED2(Rs[t0:window, t0:window]) for Rs in Rss])
print(np.nanmean(Rs_EDs))

In [ ]:
twins = [(0, 450), (100, 300), (100, 350)]
for t0, window in twins:
    Rs_EDs = np.array([tut.ED2(Rs[t0:window, t0:window]) for Rs in Rss])

    fig, ax = plt.subplots(1, 1, figsize=(2,2))
    customp = sns.color_palette('Dark2', 3)
    customp[2], customp[1] = customp[1], customp[2]
    
    # build long-form dataframe so seaborn can handle stats + errorbars
    rows = []
    conditions = ['top', 'all', 'shuff']
    rows.append({'cond': conditions[0], 'ed': tut.ED2(Rt[t0:window, t0:window]), 'kind': 'test'})
    for v in Rs_EDs:
        rows.append({'cond': conditions[2], 'ed': v, 'kind': 'shuffle'})
    rows.append({'cond': conditions[1], 'ed': tut.ED2(Ra[t0:window, t0:window]), 'kind': 'all'})
    df = pd.DataFrame(rows)   
    
    sns.barplot(data=df, x='cond', y='ed', order=conditions, hue='cond',
        estimator=np.median, errorbar='sd', err_kws={'color': 'black', 'lw': 1}, palette=customp, ax=ax, zorder=1)
    # dots for the shuffle distribution
    sns.stripplot(data=df[df['cond'] == 'shuff'], x='cond', y='ed', order=conditions, 
                  marker='.', color='black', jitter=0, alpha=0.1, ax=ax, zorder=2)
    sns.despine(fig=fig, trim=True, offset=5)

    ax.set_xticklabels([])
    # ax.set_ylim(top=20)
    ax.set_ylabel('Eff. dimensionality')
    ax.set_xlabel('')
    
    out_path = os.path.join(SAVE_DIR, 'optimal-k', f'{ROI}_{t0}-{window}_ed_all.png')
    plt.savefig(out_path, dpi=300, bbox_inches='tight', transparent=True)
    plt.show()

In [ ]:
twins = [(0, 450), (100, 300), (100, 350)]
for t0, window in twins:
    for mod in ['top', 'all']:
        Rn = Rt if 'top' in mod else Ra
        fig, ax = plt.subplots(1, 1, figsize=(2,2))
        customp = sns.color_palette('rocket', as_cmap=True)
        vmin, vmax = 0, 1 # np.nanmin(Rt), np.nanmax(Rt)

        sns.heatmap(Rn[t0:window, t0:window], vmax=vmax, vmin=vmin,
                    cbar=False, square=True,# ax=ax)
                    cmap=customp, ax=ax)
        ax.set_xticks([])
        ax.set_yticks([])
        
        ax.set_xlabel('Time')
        ax.set_ylabel('Time')
    
        out_path = os.path.join(SAVE_DIR, 'optimal-k', f'{ROI}_{t0}-{window}_{mod}.png')
        plt.savefig(out_path, dpi=300, bbox_inches='tight', transparent=True)

# # square bounds (in data / index coordinates)
# x0, x1 = 100, 350

# rect = Rectangle(
#     (x0, x0),          # bottom-left corner
#     x1 - x0,           # width
#     x1 - x0,           # height
#     fill=False,
#     edgecolor='red',
#     linewidth=2
# )

# ax.add_patch(rect)



# ax.set_title(f'Top {scale}')



# ax = axes[1]
# sns.heatmap(Ra, vmax=vmax, vmin=vmin,
#             cbar=False, square=True,# ax=ax)
#             cmap=customp, ax=ax)
# ax.set_axis_off()
# ax.set_title('Global Span')

In [ ]:
ROI = 'Unknown_19_F'  # Unknown_19_F, MF1_7_F, MF1_8_F, MF1_9_F, AF3_18_F, MB1_3_B
MODE = 'top'
scale = SCALES[ROI]

T = 450
W_step = 20
# choose ranges (edit step sizes if you want finer/slower)
t0_vals = np.arange(0, 350, 10)           # start times
win_vals = np.arange(100, 300, W_step)         # window sizes (must keep t0+win <= T)

ED = np.full((len(t0_vals), len(win_vals)), np.nan, dtype=float)

for i, t0 in enumerate(tqdm(t0_vals, desc='t0')):
    for j, win in enumerate(win_vals):
        t1 = t0 + win
        if t1 > T:
            continue
        try:
            Rt, _ = tut.static_rdm(dat, ROI, mode=MODE, scale=scale, tstart=int(t0), tend=int(t1))
            ED[i, j] = tut.ED2(Rt)
        except Exception:
            # if a window fails (e.g., too few units/images), leave nan
            continue

In [ ]:
# ED = ED.T
    
fig,ax = plt.subplots(1,1)
ax.set_xlabel('t0')
customp = sns.color_palette('Reds_r', len(ED))

for i, t0 in enumerate(ED):
    sns.lineplot(t0, alpha=0.75, color=customp[i], label=f'{i}', ax=ax)

ax.legend(title='Window size')
plt.show()

In [ ]:
# heatmap: ed(t0, window)
fig, ax = plt.subplots(1, 1)
sns.heatmap(
    ED,
    ax=ax,
    square=True,
    cmap='viridis',
    xticklabels=win_vals,
    yticklabels=t0_vals
)
ax.set_title(f'ED2 over t0 × window | ROI={ROI} | mode={MODE}')
plt.tight_layout()
plt.show()

In [ ]:
for _r, scale in SCALES.items():
    order = local_scale(dat, _r, mode=MODE, scale=scale,
                      tstart=t0, tend=t0 + window)
    
    n = np.min([scale, 20])
    ncols = 5
    nrows = int(np.ceil(n / ncols))
    
    fig, axes = plt.subplots(nrows, ncols, figsize=(ncols+1,nrows))
    axes = np.atleast_1d(axes).ravel()
    
    for ax, idx in zip(axes, order[:n]):
        fname = idx_to_fname(int(idx))
        img = Image.open(os.path.join(IMG_DIR, fname))
        ax.imshow(img, cmap='gray')
        ax.set_title(f'image {idx}', fontsize=8)
        ax.axis('off')
    
    # hide unused axes
    for ax in axes[n:]:
        ax.axis('off')
    
    fig.suptitle(f'{_r} top images')
    plt.tight_layout()
    plt.show()

In [ ]:
### PLOT in specific window
for _r in ROI_LIST:
    for _m in ['top', 'shuff']:
        scale = SCALES[_r]
        vmin, vmax= 0, 1
        Rt, _ = tut.static_rdm(dat, _r, mode=_m, scale=scale,
                          tstart=100, tend=350)
        
        fig, ax = plt.subplots(figsize=(2, 2))
        sns.heatmap(Rt, ax=ax, square=True, 
                    vmin=vmin, vmax=vmax, cbar=False)
        
        ax.set_xticks([])
        ax.set_yticks([])
        
        ax.set_xlabel('Time')
        ax.set_ylabel('Time')
        
        plt.savefig(os.path.join(SAVE_DIR, 'window', f'{_r}_{_m}.png'), format='png', dpi=300, transparent=True, bbox_inches='tight')
        plt.show()

In [ ]:
Vt = tut.rdv(Rt)
Va = tut.rdv(Ra)

# collect effective dimensionalities
Rs_EDs = np.array([tut.ED2(Rs) for Rs in Rss])

conditions = [f'T{scale}', 'S30', 'All']

# build long-form dataframe so seaborn can handle stats + errorbars
rows = []

rows.append({'cond': conditions[0], 'ed': tut.ED2(Rt), 'kind': 'test'})
for v in Rs_EDs:
    rows.append({'cond': conditions[1], 'ed': v, 'kind': 'shuffle'})
rows.append({'cond': conditions[2], 'ed': tut.ED2(Ra), 'kind': 'all'})

df = pd.DataFrame(rows)

fig, ax = plt.subplots(1, 1, figsize=(5, 5))

# bars (median + sd errorbars)
sns.barplot(
    data=df,
    x='cond',
    y='ed',
    order=conditions,
    estimator=np.median,
    errorbar='sd', errcolor='black',
    palette=['red', 'blue', 'gray'],
    ax=ax,
    zorder=1
)

# dots for the shuffle distribution (middle bar only)
sns.stripplot(
    data=df[df['cond'] == conditions[1]],
    x='cond',
    y='ed',
    order=conditions,
    color='black',
    size=2,
    jitter=0.2,
    alpha=0.1,
    ax=ax,
    zorder=2
)

ax.set_title(f'{ROI} dynamics ({t0}-{t0+window} msec)')
ax.set_xlabel('image manifold scale')
ax.set_ylabel('effective dimensionality')

sns.despine(fig, trim=True, offset=5)
plt.tight_layout()

In [ ]:
for ROI, SC in mins.items():
# for ROI in ['MB3_12_B', 'AB3_12_B', 'AB3_17_B', 'Unknown_27_B']: # not converging for: 'MB1_8_B', 'AB3_18_B',
    # SC = mins[ROI]
    scale = SC[0]
    
    MODE = 'top'
    # 100 t0, 200 window for medial
    # 150 to, 300 window for anterior
    t0 = 100 
    window = 200 #
    
    Rt, _ = static_rdm(dat, ROI, mode=MODE, scale=scale,
                      tstart=t0, tend=t0 + window)
    
    Ra, _ = static_rdm(dat, ROI, mode=MODE, scale=1072,
                      tstart=t0, tend=t0 + window)
    
    Rss = []
    for i in tqdm(range(500)):
        Rs, _ = static_rdm(dat, ROI, mode='shuff', scale=scale,
                          tstart=t0, tend=t0 + window, random_state=i)
        Rss.append(Rs)
    
    Vt = rdv(Rt)
    Va = rdv(Ra)

    try:
        # collect effective dimensionalities
        Rs_EDs = np.array([ED2(Rs) for Rs in Rss])
    except:
        Rs_EDs = [np.nan for _ in Rss]
        print(f'eigenvalues do not converge for ROI: {ROI}')
        
    conditions = [f'T{scale}', 'S30', 'All']
    
    # build long-form dataframe so seaborn can handle stats + errorbars
    rows = []
    
    rows.append({'cond': conditions[0], 'ed': ED2(Rt), 'kind': 'test'})
    for v in Rs_EDs:
        rows.append({'cond': conditions[1], 'ed': v, 'kind': 'shuffle'})
    rows.append({'cond': conditions[2], 'ed': ED2(Ra), 'kind': 'all'})
    
    df = pd.DataFrame(rows)
    
    fig, ax = plt.subplots(1, 1, figsize=(5, 5))
    
    # bars (median + sd errorbars)
    sns.barplot(
        data=df,
        x='cond',
        y='ed',
        order=conditions,
        estimator=np.median,
        errorbar='sd', errcolor='black',
        palette=['red', 'blue', 'gray'],
        ax=ax,
        zorder=1
    )
    
    # dots for the shufsfle distribution (middle bar only)
    sns.stripplot(
        data=df[df['cond'] == conditions[1]],
        x='cond',
        y='ed',
        order=conditions,
        color='black',
        size=2,
        jitter=0.2,
        alpha=0.1,
        ax=ax,
        zorder=2
    )
    
    ax.set_title(f'{ROI} dynamics ({t0}-{t0+window} msec)')
    ax.set_xlabel('image manifold scale')
    ax.set_ylabel('effective dimensionality')
    
    sns.despine(fig, trim=True, offset=5)
    plt.tight_layout()
    plt.show()